In [10]:
from utils import profiler, reader
import re
from typing import List

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [19]:
datafile = "../data/day4_input.txt"
data = reader.read_from_file(datafile)


'SXSAMASASMSXMSMSMSSMMMMMMXAMXMMXMASMMMMAXXAAAMMMSSSSSMASXXAAXASMSXXMXSXSXSASMSMMSMSAMMMAMXAAMASXMMSXMXMMMXASAXMSAASAMXSAAMXSXMASAXMMXSMSMMAM\n'

First I want to do clean up the newline characters

In [26]:
data = [x.rstrip() for x in data]

In [29]:
print(len(data), len(data[0]))

140 140


Note that the wordsearch grid is a perfect square.

# Part 1

### Overview 

Find all the word search results of XMAS

### Approach

We need to go point by point. We can't use a depth first search because that would find words going in different directions!
We have to manually scroll through all 8 directions for the word "XMAS". 

In [59]:
@profiler.profile
def part1(data: List[List[str]]) -> int:
    n = len(data)
    
    num_xmas = 0

    def scan_horizontal(x: int, y: int, length = 4) -> List[str]:
        left_string = right_string = ""
        
        for i in range(length):
            if y - i >= 0:
                left_string += data[x][y-i]
            if y + i < n:
                right_string += data[x][y+i]
        
        return [left_string, right_string]
        
    def scan_vertical(x: int, y: int, length = 4) -> List[str]:
        top_string = bottom_string = ""

        for i in range(length):
            if x - i >= 0:
                top_string += data[x-i][y]
            if x + i < n:
                bottom_string += data[x+i][y]

        return [top_string, bottom_string]
    
    def scan_diagonal(x: int, y: int, length = 4) -> List[str]:
        #There are four directions to scan

        top_left = top_right = bottom_left = bottom_right = ""

        for i in range(length):
            if x - i >= 0 and y - i >= 0: #Within bounds going up and left
                top_left += data[x-i][y-i]
            if x - i >= 0 and y + i < n: #Within bounds going up and right
                top_right += data[x-i][y+i]
            if x + i < n and y - i >= 0: #Within bounds going down and left
                bottom_left += data[x+i][y-i]
            if x + i < n and y + i < n: #Within bounds going down and right
                bottom_right += data[x+i][y+i]
        
        return [top_left, top_right, bottom_left, bottom_right]
            

    for i in range(n):
        for j in range(n):
            words_starting_at_ij = scan_diagonal(i,j) + scan_horizontal(i,j) + scan_vertical(i,j)
            for word in words_starting_at_ij:
                if word == "XMAS":
                    num_xmas += 1

    return num_xmas

part1(data)
    

Calling part1: Memory used 319488 kB; Execution Time: 0.121974834240973 s


2401

# Part 2

### Overview 

We want to find the letters "MAS" in the shape of an X like so:

"M"."S"

. "A" .

"M"."S"

### Approach

We can scan through and search the diagonals of ONLY A's since A must be at the center of each instance. This will reduce the amount of checking we have to do. Then we look one to the top left, top right, bottom left, and bottom right to see if this forms a valid x-mas.
We can check that the diagonals to form either MAS or MAS[::-1] = SAM for them to be valid. Both diagonals must be valid for this to count as an x-mas

In [68]:
@profiler.profile
def part2(data: List[str]) -> int:
    num_x_mas = 0
    n = len(data)
    valid = ('SAM', 'MAS')
    def scan_diagonals(x: int, y: int) -> bool:
        diagonal_1 = data[x-1][y-1] + data[x][y] + data[x+1][y+1] #top left to bottom right
        diagonal_2 = data[x-1][y+1] + data[x][y] + data[x+1][y-1] #top right to bottom left
        return (diagonal_1 in valid) and (diagonal_2 in valid)
            
    for i in range(1, n - 1):
        for j in range(1, n - 1):
            if data[i][j] == 'A':
                num_x_mas += scan_diagonals(i, j) #since this returns a boolean, we can use truthy/falsey values.

    return num_x_mas

part2(data)
        


Calling part2: Memory used 0 kB; Execution Time: 0.005340042058378458 s


1822